### Introduction
This notebook assumes that metronome service is already running locally on default port 5000 and is ready to receive HTTP calls. It will not work if the service is not running.
I will present a series of time points and then try to predict whether there is a pattern of points repeating every 5 minutes. 

### Data generation
Firstly, I will generate the series of time points repeating every 5 minutes and at the same time inject some additional random data points into it.

In [1]:
import pandas as pd 
import numpy as np 
import json 
import requests

np.random.seed(seed=11) # so the results will be reproducible
np.set_printoptions(precision=3)

interval = 300 # in seconds; equal to 5 minutes
differences = []


number_of_points_in_pattern = 30
times = []
labels = []

last_pattern = 0
for _ in range(number_of_points_in_pattern):
    additional_records_number = int(np.random.uniform(low=0, high=5))   # number of points in the noise
    additional_records_times = list(np.sort(np.random.uniform(low=5, high=295, size=additional_records_number)) + last_pattern)
    last_pattern = last_pattern + interval + int(np.random.uniform(low=-3, high=3)) # let's have some randomness to the pattern

    times = times + additional_records_times
    times.append(last_pattern)

    part_of_pattern = [0] * additional_records_number # 0s for noise
    labels = labels + part_of_pattern
    labels.append(1)    # for the one point from pattern

times_and_labels = pd.DataFrame({ "times": times, "labels": labels})
times_and_labels.head()

,times,labels
0,298.000000,1
1,424.859045,0
2,513.230839,0
3,598.000000,1
4,898.000000,1


This is the pattern that we will try to find in the data. Let us calculate differences between those points. Differences is the input for Metronome API.

In [2]:
differences = np.diff(times_and_labels["times"])
print("First 10 differences in seconds: {}".format(differences[:10]))

First 10 differences in seconds: [126.859  88.372  84.769 300.     36.533 180.156  35.041  12.502  37.768
 300.   ]


### Querying the server
The input to the API needs to be a json structure with 15 variables - 14 differences between points and interval key. We will try to predict whether a single point is a part of the pattern.

In [17]:
point_to_predict = times_and_labels.iloc[15,:]  # let us get predictions and probabilities about the 15th point
print("Point to predict: \n{}".format(point_to_predict))

def prepare_request(time_differences, interval):
    payload = {"var" + str(i) : difference for i, difference in enumerate(time_differences)}
    payload["interval"] = interval

    return payload

payload = json.dumps(prepare_request(differences[:14], 5))
print("JSON content of the POST request:\n{}".format(payload))

Point to predict: 
times     2136.381757
labels       0.000000
Name: 15, dtype: float64
JSON content of the POST request:
{"var0": 126.85904533044095, "var1": 88.3717941352819, "var2": 84.76916053427715, "var3": 300.0, "var4": 36.53346084200268, "var5": 180.15623552203476, "var6": 35.04087955522846, "var7": 12.501633463407643, "var8": 37.767790617326455, "var9": 300.0, "var10": 298.0, "var11": 50.794568935731604, "var12": 250.2054310642684, "var13": 29.346411563396487, "interval": 5}


Now, assuming the Metronome microservice is running on the default port 5000, we can use requests to query the service. The response from the server is in JSON, so it needs to be loaded first. 

In [16]:
response = requests.post("http://127.0.0.1:5000/v1/models/metronome", json=payload)
print("Metronome response:\n{}".format(response.text))

probabilities_and_predictions = json.loads(response.text)

Metronome response:
{"probabilities": [1.47523243620995e-10], "predictions": [0]}


### Batch requests
It is possible to send more than one record at a time by increasing the number of elements in the payload dictionary's lists. Here is an example:

In [35]:
from collections import defaultdict

points_to_predict = times_and_labels.iloc[15:17, :]
print("Points to predict:\n{}".format(points_to_predict))

first_element = prepare_request(differences[:14], 5)
second_element = prepare_request(differences[1:15], 5)

batch_request = defaultdict(list)
for single_request in (first_element, second_element):
    for key, value in single_request.items():
        batch_request[key].append(value)

batch_request["interval"] = 5

batch_payload = json.dumps(batch_request)
print("Batch of two points to fit:\n{}".format(batch_payload))

batch_response = requests.post("http://127.0.0.1:5000/v1/models/metronome", json=batch_payload)
batch_probas_and_predictions = json.loads(batch_response.text)
print("Batch probabilities and predictions:\n{}".format(batch_probas_and_predictions))

Points to predict:
          times  labels
15  2136.381757       0
16  2196.451651       0
Batch of two points to fit:
{"var0": [126.85904533044095, 88.3717941352819], "var1": [88.3717941352819, 84.76916053427715], "var2": [84.76916053427715, 300.0], "var3": [300.0, 36.53346084200268], "var4": [36.53346084200268, 180.15623552203476], "var5": [180.15623552203476, 35.04087955522846], "var6": [35.04087955522846, 12.501633463407643], "var7": [12.501633463407643, 37.767790617326455], "var8": [37.767790617326455, 300.0], "var9": [300.0, 298.0], "var10": [298.0, 50.794568935731604], "var11": [50.794568935731604, 250.2054310642684], "var12": [250.2054310642684, 29.346411563396487], "var13": [29.346411563396487, 8.035345715054973], "interval": 5}
Batch probabilities and predictions:
{'probabilities': [1.4752014887431386e-10, 5.759364398727485e-07], 'predictions': [0, 0]}
